In [1]:
import os
import datajoint as dj
dj.config['database.host'] = os.environ['DJ_HOST']
dj.config['database.user'] = os.environ['DJ_USERNAME']
dj.config['database.password'] = os.environ['DJ_PASSWORD']
dj.config['enable_python_native_blobs'] = True
dj.config['schema_name'] = "nnfabrik_example"

from nnfabrik.main import *
import nnfabrik as nnf

# nnf.config['repos'] = ['/notebooks/lib/nnfabrik']

Connecting mohammad@datajoint-db.mlcloud.uni-tuebingen.de:3306


In [17]:
schema.drop()

Proceed to delete entire schema `nnfabrik_example`? [yes, No]:  yes


In [2]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,email e-mail address,affiliation conributor's affiliation,dj_username DataJoint username


In [3]:
aa = dict(fabrikant_name="Mohammad Bashiri", email="mohammadbashiri93@gmail.com", affiliation="sinzlab", dj_username="mohammad")
Fabrikant().insert1(aa)

In [4]:
Fabrikant()

fabrikant_name Name of the contributor that added this entry,email e-mail address,affiliation conributor's affiliation,dj_username DataJoint username
Mohammad Bashiri,mohammadbashiri93@gmail.com,sinzlab,mohammad


## Dataset table

In [5]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion


In [6]:
dataset_config = dict(seed=1, param1=10)
dataset_entry = dict(dataset_fn="nnfabrik.datasets.toy_datasets.toy_dataset", dataset_config=dataset_config, dataset_comment="A comment about dataset!")
Dataset().add_entry(**dataset_entry)

In [7]:
Dataset()

dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,dataset_config dataset configuration object,dataset_fabrikant Name of the contributor that added this entry,dataset_comment short description,dataset_ts UTZ timestamp at time of insertion
nnfabrik.datasets.toy_datasets.toy_dataset,38b3eff8baf56627478ec76a704e9b52,=BLOB=,Mohammad Bashiri,A comment about dataset!,2019-12-11 12:49:27


In [8]:
model_config= dict(kern=16, kern2=64)
model_entry = dict(model_fn="nnfabrik.models.toy_models.toy_model", model_config=model_config, model_fabrikant="Mohammad Bashiri", model_comment="Another awesome model")
Model().add_entry(**model_entry)

In [9]:
Model()

model_fn name of the model function,model_hash hash of the model configuration,model_config model configuration to be passed into the function,model_fabrikant Name of the contributor that added this entry,model_comment short description,model_ts UTZ timestamp at time of insertion
nnfabrik.models.toy_models.toy_model,4f398cb9d6bc79ae567298335b51ba8a,=BLOB=,Mohammad Bashiri,Another awesome model,2019-12-11 12:49:27


In [10]:
trainer_config= dict(param1=3)
trainer_entry = dict(trainer_fn="nnfabrik.training.toy_trainers.toy_trainer", trainer_config=trainer_config, trainer_fabrikant="Mohammad Bashiri", trainer_comment="Of course with an increadible trainer")
Trainer().add_entry(**trainer_entry)

In [11]:
Trainer()

trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,trainer_config training configuration object,trainer_fabrikant Name of the contributor that added this entry,trainer_comment short description,trainer_ts UTZ timestamp at time of insertion
nnfabrik.training.toy_trainers.toy_trainer,eccbc87e4b5ce2fe28308fd9f2a7baf3,=BLOB=,Mohammad Bashiri,Of course with an increadible trainer,2019-12-11 12:49:27


In [12]:
TrainedModel()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion


In [13]:
Seed()

seed Random seed that is passed to the model- and dataset-builder


In [14]:
Seed().insert1({"seed": 1})

In [15]:
Seed()

seed Random seed that is passed to the model- and dataset-builder
1


In [16]:
TrainedModel().populate(display_progress=True)

100%|██████████| 1/1 [00:00<00:00, 13.07it/s]


In [17]:
TrainedModel()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,score loss,output trainer object's output,fabrikant_name Name of the contributor that added this entry,trainedmodel_ts UTZ timestamp at time of insertion
nnfabrik.models.toy_models.toy_model,4f398cb9d6bc79ae567298335b51ba8a,nnfabrik.datasets.toy_datasets.toy_dataset,38b3eff8baf56627478ec76a704e9b52,nnfabrik.training.toy_trainers.toy_trainer,eccbc87e4b5ce2fe28308fd9f2a7baf3,1,1.0,=BLOB=,Mohammad Bashiri,2019-12-11 12:49:30


In [18]:
TrainedModel().GitLog()

model_fn name of the model function,model_hash hash of the model configuration,dataset_fn name of the dataset loader function,dataset_hash hash of the configuration object,trainer_fn name of the Trainer loader function,trainer_hash hash of the configuration object,seed Random seed that is passed to the model- and dataset-builder,info


In [19]:
TrainedModel().GitLog().fetch("info")

array([], dtype=object)